In [1]:
# import pytorch libraries
%matplotlib inline
import torch 
import torch.autograd as autograd 
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np

# CBOW model for text classification
In this part of the tutorial we develop a continuous bag of words (CBOW) model for a text classification task described [here]( https://people.cs.umass.edu/~miyyer/pubs/2015_acl_dan.pdf). The CBOW model was first described [here](https://arxiv.org/pdf/1301.3781.pdf)

## Subjectivity Dataset
The subjectivity dataset has 5000 subjective and 5000 objective processed sentences. To get the data:
```
wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
```

In [2]:
def unpack_dataset():
    ! wget http://www.cs.cornell.edu/people/pabo/movie-review-data/rotten_imdb.tar.gz
    ! mkdir data
    ! tar -xvf rotten_imdb.tar.gz -C data

In [3]:
#unpack_dataset()

In [4]:
!ls data

plot.tok.gt9.5000   quote.tok.gt9.5000  subjdata.README.1.0


In [5]:
! head -2 data/plot.tok.gt9.5000

the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . 
emerging from the human psyche and showing characteristics of abstract expressionism , minimalism and russian constructivism , graffiti removal has secured its place in the history of modern art while being created by artists who are unconscious of their artistic achievements . 


In [6]:
from pathlib import Path
PATH = Path("data")
list(PATH.iterdir())

[PosixPath('data/plot.tok.gt9.5000'),
 PosixPath('data/subjdata.README.1.0'),
 PosixPath('data/quote.tok.gt9.5000')]

## Tokenization
Tokenization is the task of chopping up text into pieces, called tokens.

spaCy is an open-source software library for advanced Natural Language Processing. Here we will use it for tokenization.  

### Simple Tokenization

In [7]:
# We need each line in the file 
def read_file(path):
    """ Read file returns a list of lines.
    """
    with open(path, encoding = "ISO-8859-1") as f:
        content = f.readlines()
    return content

In [8]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [9]:
obj_lines[0]

'the movie begins in the past where a young boy named sam attempts to save celebi from a hunter . \n'

In [10]:
np.array(obj_lines[0].strip().lower().split(" "))

array(['the', 'movie', 'begins', 'in', 'the', 'past', 'where', 'a',
       'young', 'boy', 'named', 'sam', 'attempts', 'to', 'save', 'celebi',
       'from', 'a', 'hunter', '.'], dtype='<U8')

### Much better tokenization with Spacy

In [ ]:
#!pip install -U spacy

In [13]:
#import spacy

In [ ]:
# first time run this
#!python3 -m spacy download en

In [ ]:
tok = spacy.load('en')

In [ ]:
obj_lines = read_file(PATH/"plot.tok.gt9.5000")

In [ ]:
len(obj_lines)

In [ ]:
obj_lines[0]

In [ ]:
test = tok(obj_lines[0])

In [ ]:
np.array([x for x in test])

## Split dataset in train and test

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
sub_content = read_file(PATH/"quote.tok.gt9.5000")
obj_content = read_file(PATH/"plot.tok.gt9.5000")
sub_content = np.array([line.strip().lower() for line in sub_content])
obj_content = np.array([line.strip().lower() for line in obj_content])
sub_y = np.zeros(len(sub_content))
obj_y = np.ones(len(obj_content))
X = np.append(sub_content, obj_content)
y = np.append(sub_y, obj_y)

In [13]:
X[0], y[0]

('smart and alert , thirteen conversations about one thing is a small gem .',
 0.0)

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train[:5], y_train[:5]

(array(['will god let her fall or give her a new path ?',
        "the director's twitchy sketchbook style and adroit perspective shifts grow wearisome amid leaden pacing and indifferent craftsmanship ( most notably wretched sound design ) .",
        "welles groupie/scholar peter bogdanovich took a long time to do it , but he's finally provided his own broadside at publishing giant william randolph hearst .",
        'based on the 1997 john king novel of the same name with a rather odd synopsis : " a first novel about a seasoned chelsea football club hooligan who represents a disaffected society operating by brutal rules .',
        'yet , beneath an upbeat appearance , she is struggling desperately with the emotional and physical scars left by the attack .'],
       dtype='<U691'), array([1., 0., 0., 1., 1.]))

## Word to index mapping
Here we will compute a vocabulary of words based on the training set and a mapping from word to an index.

In [16]:
from collections import defaultdict

In [17]:
def get_vocab(content):
    """Computes Dict of counts of words.
    
    Computes the number of times a word is on a document.
    """
    vocab = defaultdict(float)
    for line in content:
        words = set(line.split())
        for word in words:
            vocab[word] += 1
    return vocab      

In [18]:
#Getting the vocabulary from the training set
word_count = get_vocab(X_train)

In [60]:
#word_count

In [20]:
len(word_count.keys())

21415

In [21]:
# let's delete words that are very infrequent
for word in list(word_count):
    if word_count[word] < 5:
        del word_count[word]

In [22]:
len(word_count.keys())

4065

In [23]:
## Finally we need an index for each word in the vocab
vocab2index = {"<PAD>":0, "UNK":1} # init with padding and unknown
words = ["<PAD>", "UNK"]
for word in word_count:
    vocab2index[word] = len(words)
    words.append(word)

In [62]:
#vocab2index

## Sentence encoding
Here we encode each sentence as a sequence of indices corresponding to each word.

In [25]:
x_train_len = np.array([len(x.split()) for x in X_train])
x_valid_len = np.array([len(x.split()) for x in X_valid])

In [26]:
np.percentile(x_train_len, 95) # let set the max sequence len to N=40

43.0

In [27]:
X_train[0]

'will god let her fall or give her a new path ?'

In [28]:
# returns the index of the word or the index of "UNK" otherwise
vocab2index.get("?", vocab2index["UNK"])

5

In [29]:
np.array([vocab2index.get(w, vocab2index["UNK"]) for w in X_train[0].split()])

array([ 7, 11,  6,  8, 12,  9,  4,  8, 10,  2,  3,  5])

In [30]:
def encode_sentence(s, N=40):
    enc = np.zeros(N, dtype=np.int32)
    enc1 = np.array([vocab2index.get(w, vocab2index["UNK"]) for w in s.split()])
    l = min(N, len(enc1))
    enc[:l] = enc1[:l]
    return enc, l

In [31]:
encode_sentence(X_train[0])

(array([ 7, 11,  6,  8, 12,  9,  4,  8, 10,  2,  3,  5,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0], dtype=int32), 12)

## Dataset

In [32]:
class SubjectivityDataset(Dataset):
    def __init__(self, X, y):
        self.x = X
        self.y = y
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x = self.x[idx]
        x, s = encode_sentence(x)
        return x, self.y[idx], s
    
train_ds = SubjectivityDataset(X_train, y_train)
valid_ds = SubjectivityDataset(X_valid, y_valid)

In [33]:
batch_size=3
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=batch_size)

In [34]:
x, y, s = next(iter(train_dl))

In [35]:
x

tensor([[1189,   55,   13, 1190,  137,  440,  492,  140,    1,   30,  431, 3393,
         3221,   55,    1,   81,    8,  175,   23,  434, 1141, 2166,  443,   76,
          408,  105, 1555,   40,  552,   26,  946,  100, 2289,  104,    1,   14,
            0,    0,    0,    0],
        [2477,    1,  627,   10,  620,   23,    1,    1, 1816, 2478,    1,   14,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0],
        [ 212,   10,  650,  295,   55,    1,   23,  640,  105,    1,   14,   40,
          214,  585,    1,   32,   70, 1139,   14,   14,   14,   81,  241,    1,
          126,    1,   70,    1,   14,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]], dtype=torch.int32)

In [36]:
y

tensor([0., 0., 0.])

In [37]:
# lenght of each vector
s

tensor([36, 12, 29])

## Embedding layer
Most deep learning models use a dense vectors of real numbers as representation of words (word embeddings), as opposed to a one-hot encoding representations. The module torch.nn.Embedding is used to represent word embeddings. It takes two arguments: the vocabulary size, and the dimensionality of the embeddings. The embeddings are initialized with random vectors. 

In [38]:
# an Embedding module containing 10 words with embedding size 4
# embedding will be initialized at random
embed = nn.Embedding(10, 4, padding_idx=0)
embed.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.4715, -0.4786,  0.3576, -1.5607],
        [-0.7653, -1.8298,  0.1394, -0.4841],
        [-1.5639,  1.0803, -0.0774,  1.4944],
        [-0.0751,  0.2259, -0.3835,  0.3063],
        [ 0.1710,  0.3455, -0.1925, -1.2649],
        [-1.1988,  0.0703, -0.6465, -0.5690],
        [ 0.2810,  0.7461,  0.0589, -0.1314],
        [ 0.7216,  0.8877,  0.0697, -0.0167],
        [-0.4183,  0.5204, -0.2750, -1.6212]], requires_grad=True)

Note that the `padding_idx` has embedding vector 0.

In [39]:
# given a list of ids we can "look up" the embedding corresponing to each id
# can you see that some vectors are the same?
a = torch.LongTensor([[1,4,1,5,1,0]])
embed(a)

tensor([[[ 1.4715, -0.4786,  0.3576, -1.5607],
         [-0.0751,  0.2259, -0.3835,  0.3063],
         [ 1.4715, -0.4786,  0.3576, -1.5607],
         [ 0.1710,  0.3455, -0.1925, -1.2649],
         [ 1.4715, -0.4786,  0.3576, -1.5607],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

This would be the representation of a sentence with words with indices [1,4,1,5,1] and a padding at the end. Bellow we have an example in which we have two sentences. the first sentence has length 3 and the last sentence has length 2. In order to use a tensor we use padding at the end of the second sentence. 

In [40]:
a = torch.LongTensor([[1,4,1], [1,3,0]])

Our model takes an average of the word embedding of each word. Here is how we do it.

In [41]:
s = torch.FloatTensor([3, 2]) # here is the size of the vector

In [42]:
embed(a)

tensor([[[ 1.4715, -0.4786,  0.3576, -1.5607],
         [-0.0751,  0.2259, -0.3835,  0.3063],
         [ 1.4715, -0.4786,  0.3576, -1.5607]],

        [[ 1.4715, -0.4786,  0.3576, -1.5607],
         [-1.5639,  1.0803, -0.0774,  1.4944],
         [ 0.0000,  0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward>)

In [43]:
embed(a).sum(dim=1)

tensor([[ 2.8679, -0.7313,  0.3317, -2.8150],
        [-0.0924,  0.6017,  0.2802, -0.0663]], grad_fn=<SumBackward1>)

In [44]:
sum_embs = embed(a).sum(dim=1) 
sum_embs/ s.view(s.shape[0], 1)

tensor([[ 0.9560, -0.2438,  0.1106, -0.9383],
        [-0.0462,  0.3009,  0.1401, -0.0332]], grad_fn=<DivBackward0>)

## Continuous Bag of Words Model

In [70]:
class CBOW(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear1 = nn.Linear(emb_size, 30)
        self.linear2 = nn.Linear(30, 1)
        
    def forward(self, x, s):
        x = self.word_emb(x)
        x = x.sum(dim=1)/ s.view(s.shape[0], 1)
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        return x

In [71]:
model = CBOW(vocab_size=5, emb_size=3)

In [72]:
model.word_emb.weight

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000],
        [-0.0330, -0.5634, -0.8898],
        [-0.6565, -0.9158, -1.1859],
        [-1.0056, -0.5797, -1.3202],
        [ 1.9810, -0.2791,  0.0940]], requires_grad=True)

In [48]:
model(a, s)

tensor([[ 0.2648],
        [-0.1347]], grad_fn=<AddmmBackward>)

# Training the CBOW model 

In [49]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
print(V)

4067


In [64]:
batch_size=500
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=2000)

In [65]:
len(valid_ds)

2000

In [66]:
def test_metrics(model):
    model.eval()
    for x, y, s in valid_dl:
        s = torch.FloatTensor(s.float()).view(s.shape[0], 1)
        y = y.unsqueeze(1)
        y_hat = model(x.long(), s)
        loss = F.binary_cross_entropy_with_logits(y_hat, y)
        y_pred = y_hat > 0
        correct = (y_pred.float() == y).float().sum()
        accuracy = correct/y_pred.shape[0]
    return loss.item(), accuracy.item()

In [67]:
# accuracy of a random model should be around 0.5
test_metrics(model)

(0.2677851915359497, 0.8870000243186951)

In [68]:
def train_epocs(model, epochs=10, lr=0.01, weight_decay=1e-5):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    for i in range(epochs):
        model.train()
        for x, y, s in train_dl:
            y = y.unsqueeze(1)
            s = s.type(torch.Tensor).view(s.shape[0], 1)
            y_hat = model(x.long(), s)
            loss = F.binary_cross_entropy_with_logits(y_hat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        val_loss, val_acc = test_metrics(model)
        print("train loss %.3f val loss %.3f and val accuracy %.3f" % (loss.item(), val_loss, val_acc))

In [73]:
V = len(words)
model = CBOW(vocab_size=V, emb_size=50)
train_epocs(model, epochs=15)

train loss 0.553 val loss 0.522 and val accuracy 0.760
train loss 0.321 val loss 0.332 and val accuracy 0.860
train loss 0.212 val loss 0.278 and val accuracy 0.896
train loss 0.147 val loss 0.269 and val accuracy 0.896
train loss 0.134 val loss 0.281 and val accuracy 0.897
train loss 0.084 val loss 0.302 and val accuracy 0.892
train loss 0.067 val loss 0.343 and val accuracy 0.890
train loss 0.040 val loss 0.387 and val accuracy 0.882
train loss 0.043 val loss 0.424 and val accuracy 0.879
train loss 0.037 val loss 0.460 and val accuracy 0.874
train loss 0.023 val loss 0.509 and val accuracy 0.873
train loss 0.012 val loss 0.542 and val accuracy 0.873
train loss 0.013 val loss 0.565 and val accuracy 0.876
train loss 0.009 val loss 0.587 and val accuracy 0.873
train loss 0.005 val loss 0.607 and val accuracy 0.875


In [58]:
class Word2Vec(nn.Module):
    def __init__(self, vocab_size, emb_size=100):
        super(CBOW, self).__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.linear = nn.Linear(emb_size, vocab_size, bias=False)
        
    def forward(self, x):
        x = self.word_emb(x)
        x = self.linear(x)
        return x

In [ ]:
# This criterion combines log_softmax and nll_loss in a single function.
loss = F.cross_entropy(model(x), y)

## Lab
* Apply this model to any text classification problem. Here are a couple of problems:
    * https://ai.stanford.edu/~amaas/data/sentiment/ (sentiment classification)
    * https://www.kaggle.com/yelp-dataset/yelp-dataset
* More challenging, modify the cbow model to decide it two sentences have the same intent.
    * https://www.kaggle.com/c/quora-question-pairs

# References
* https://pytorch.org/docs/stable/index.html
* http://pytorch.org/tutorials/beginner/pytorch_with_examples.html
* https://hsaghir.github.io/data_science/pytorch_starter/